## Convert GLIMS polygons to RGI polygons: Level 1 files

GLIMS outlines are shared in a format where each polygon is either a glacier outline or a rock outcrop (nunatak). RGI outlines are provided as polygons with exterior (glacier outlines) and interior (outcrops) boundaries. GLIMS does offer download in either format, but it is buggy and [silently fails on some glaciers](https://github.com/GLIMS-RGI/glims_issue_tracker/issues/2).

Therefore, this script is there to convert the GLIMS format to RGI format. L1 files are simply the same data as L0 but with the new format. Additionally, we kept a list of "orphaned" outcrops which hint at errors in either the GLIMS database or this script.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import shutil
import glob
import os
import subprocess
import tarfile
import shapely.geometry as shpg
import progressbar
import warnings
import utils

In [2]:
# go down from rgi7_scripts/workflow/preprocessing
data_dir = '../../../rgi7_data/'

In [3]:
reg_file = os.path.join(data_dir, 'l0_regions', '00_rgi70_regions', '00_rgi70_O1Regions.shp')
reg_f = gpd.read_file(reg_file)

In [4]:
# We drop the second alaska box which is useless
reg_f = reg_f.drop(1)

In [5]:
odir = utils.mkdir(os.path.join(data_dir, 'l1_with_interiors'))
odir_tar = utils.mkdir(os.path.join(data_dir, 'l1_with_interiors_tars'))

odir_failed = utils.mkdir(os.path.join(data_dir, 'l1_orphan_interiors'))
odir_failed_tar = utils.mkdir(os.path.join(data_dir, 'l1_orphan_interiors_tars'))

# for reg in range(1, 20):
for reg in [3]:
    
    # Read the shapes for one or more boxes
    shp = [] 
    for i, sreg in reg_f.loc[reg_f.RGI_CODE == f'{reg:02d}'].iterrows():
        
        filebasename = os.path.join(data_dir, 'l0_from_glims', f'{i:02d}_RGI{reg:02d}.tgz')
        print(filebasename)
        with tarfile.open(filebasename, "r:gz") as tar:
            fn = tar.getnames()[0]
        fp = 'tar://' + filebasename + '/{}/glims_polygons.shp'.format(fn)
        shp.append(gpd.read_file(fp))
    
    # Merge them together
    shp = pd.concat(shp)    
    
    # Compute areas
    shp['area'] = shp.to_crs({'proj':'cea'}).area
    
    # Now the interior things
    
    # We group per anlys_id which is the smalles unit of glacier in GLIMS 
    # (a single GLIMSID can have several dates and/or contributor, for example)
    grouped = shp.groupby('anlys_id')
    
    # We loop over anlys_id
    odf = []
    odf_rocks = []
    for name, group in progressbar.progressbar(grouped):

        # Basic sanity checks
        assert len(group.glac_id.unique()) == 1
        assert len(group.anlys_time.unique()) == 1
        assert len(group.src_date.unique()) == 1
        assert len(group.release_dt.unique()) == 1
        assert len(group.submitters.unique()) == 1
        assert len(group.analysts.unique()) == 1

        # Select the various (1 - more) main rocks
        mains = group.loc[group.line_type == 'glac_bound'].copy()

        # And all rocks
        rocks = group.loc[group.line_type == 'intrnl_rock']

        # We sort areas to avoid misclassifications
        mains = mains.sort_values('area')

        # Now loop over the mains and check which rocks belong
        for i, main in mains.iterrows():

            try:
                # Check where the rocks belong
                isin = rocks.geometry.within(main.geometry)
            except:
                # An error occurred
                # The ids below have been manually checked - that's not many in light of the thousands of entries
                if main.glac_id in ['G282321E08973S', 'G286431E47184S', 'G293140E68314S', 
                                    'G294835E68160S', 'G295727E65746S']:
                    mains = mains.drop(i)
                    isin = []
                elif main.glac_id in ['G286765E46650S']:
                    main['geometry'] = main.geometry.buffer(0)[1]
                    isin = rocks.geometry.within(main.geometry)
                elif main.glac_id in ['G282313E08974S', 'G286697E46659S', 'G286765E46650S', 
                                      'G292617E67510S', 'G294132E67719S', 'G012723E47107N']:
                    rocks = rocks[rocks.is_valid]
                    isin = rocks.geometry.within(main.geometry)
                else:
                    raise
            exterior = main.geometry.exterior
            interiors = [p.exterior for p in rocks.loc[isin].geometry]
            if len(interiors) > 0:
                mains.loc[i, 'geometry'] = shpg.Polygon(exterior, interiors)
                rocks = rocks[~isin]

        odf.append(mains)
        odf_rocks.append(rocks)

    odf = pd.concat(odf)
    odf_rocks = pd.concat(odf_rocks)
    
    dd = utils.mkdir(f'{odir}/RGI{reg:02d}/', reset=True)
    print('Writing...')
    odf.to_file(dd + f'RGI{reg:02d}.shp')
    
    print('Taring...')
    print(subprocess.run(['tar', '-zcvf', f'{odir_tar}/RGI{reg:02d}.tar.gz', '-C', odir, f'RGI{reg:02d}']))
    
    if len(odf_rocks) > 0:
        dd = utils.mkdir(f'{odir_failed}/RGI{reg:02d}/', reset=True)
        print('Writing orphans...')
        odf_rocks.to_file(dd + f'RGI{reg:02d}.shp')

        print('Taring orphans...')
        print(subprocess.run(['tar', '-zcvf', f'{odir_failed_tar}/RGI{reg:02d}.tar.gz', '-C', odir_failed, f'RGI{reg:02d}']))

../../../rgi7_data/l0_from_glims/03_RGI03.tgz


100% (14995 of 14995) |##################| Elapsed Time: 0:01:22 Time:  0:01:22


Writing...
Taring...
CompletedProcess(args=['tar', '-zcvf', '../../../rgi7_data/l1_with_interiors_tars/RGI03.tar.gz', '-C', '../../../rgi7_data/l1_with_interiors', 'RGI03'], returncode=0)
Writing orphans...
Taring orphans...
CompletedProcess(args=['tar', '-zcvf', '../../../rgi7_data/l1_orphan_interiors_tars/RGI03.tar.gz', '-C', '../../../rgi7_data/l1_orphan_interiors', 'RGI03'], returncode=0)
